In [ ]:
import os
import json
import pickle
from itertools import chain

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
%matplotlib inline

from neuron import h
from current_injection import inject_current_step
from dlutils import utils
from dlutils.cell import Cell

In [ ]:
optimization_folder = '/Users/daniele/Postdoc/Research/CA3/OPTIMIZATIONS/'

# config_file = 'f-I_curve_thorny.json'
config_file = 'f-I_curve_a-thorny.json'

config = json.load(open(config_file, 'r'))
cell_type = config['cell_type']
prefix = cell_type[0].upper() + cell_type[1:]
base_folder = optimization_folder + prefix + '/' + config['cell_name'] + '/' + config['optimization_run'] + '/'
swc_file = config['swc_file']
individual = config['individual']

swc_file = base_folder + swc_file

params_file = base_folder + 'individual_{}.json'.format(individual)
parameters = json.load(open(params_file, 'r'))

config_file = base_folder + 'parameters.json'
cell_name = config['cell_name'] + '_'
mechanisms = utils.extract_mechanisms(config_file, cell_name)

sim_pars = pickle.load(open(base_folder + 'simulation_parameters.pkl','rb'))
replace_axon = sim_pars['replace_axon']
add_axon_if_missing = not sim_pars['no_add_axon']

In [ ]:
delay = 500
dur = 500
after = 1000
inj_loc = 'soma'
inj_dist = 0
xls_file = f'f-I_curve_{cell_type}.xlsx'
n_I = len(config['I'])
cmap = plt.get_cmap('PiYG', n_I)
fig,ax = plt.subplots(1, 1, figsize=(10,4))
fc = [.85, .85, .85]
with pd.ExcelWriter(xls_file, mode='w') as writer:
    for n,I in enumerate(config['I']):
        recorders = inject_current_step(I, delay, dur, swc_file, inj_loc, inj_dist, parameters, mechanisms,
                                        after, replace_axon=replace_axon, add_axon_if_missing=add_axon_if_missing)
        t = np.array(recorders['t'])
        Vm = np.array(recorders['soma.v'])
        idx = (t > delay - 100) & (t < delay + dur + 500)
        plt.plot(t[idx] - delay, Vm[idx], lw=2, color=cmap(n), label=f'I = {I} pA')
        df = pd.DataFrame(data = {'Time': t[idx] - delay, 'Vsoma': Vm[idx]})
        df.to_excel(writer, sheet_name=f'I = {I} pA')
ax.set_facecolor(fc)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Vm (mV)')
ax.legend(loc='upper right', facecolor=fc, frameon=False)
for side in 'right','top':
    ax.spines[side].set_visible(False)
fig.savefig(os.path.splitext(xls_file)[0] + '.pdf')